In [112]:
import verbose_module; import sys; import os
import matplotlib.pyplot as plt
import seaborn as sns; import pandas as pd
import numpy as np

from scipy.stats import ttest_rel
from pickle import load
from matplotlib.patches import Circle
from webcolors import name_to_rgb
from itertools import product
from copy import deepcopy

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('../../.')
import pIRPgym


weights = [[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]

other_path = False

In [113]:
def import_rewards(w, K, T, theta, gamma, min_q):

    rewards = dict()
    for seed in range(24):
        rewards[seed] = verbose_module.import_results.import_rewards(w,seed_ix=seed,other_path=other_path, theta = theta, gamma = gamma, min_q = min_q)

    agg_rewards = dict()
    for e in rewards[0][1]:
        agg_rewards[e] = {k:[sum(rewards[seed][t+1][e][k] for t in T) for seed in rewards] for k in K}
        agg_rewards[e]["total"] = [sum(rewards[seed][t+1][e][k] for t in T for k in K) for seed in rewards]
    
    return agg_rewards

def get_rewards_and_sets(w1,w2, theta, gamma, min_q):

    inst_gen = verbose_module.import_results.import_instance_parameters(w2, 0, other_path=other_path, theta = theta, gamma = gamma, min_q = min_q)
    E = inst_gen.E+["transport cost","purchase cost","backorders cost","holding cost"]
    K = inst_gen.Products; T = inst_gen.Horizon; I = list(range(24))
    metric_names = inst_gen.metric_names

    base = import_rewards(w1, K, T, theta = theta, gamma = gamma, min_q = min_q); policy = import_rewards(w2, K, T, theta = theta, gamma = gamma, min_q = min_q)

    return base, policy, E, K, T, I, metric_names

def compute_total_cost_differences(base, policy, E, P, I):

    tc_base = [sum(base[e][p][it] for e in E[4:] for p in P) for it in I]
    tc_policy = [sum(policy[e][p][it] for e in E[4:] for p in P) for it in I]

    return tc_base, tc_policy

def compute_relative_performance(base,policy, P, I, metrics = ["transport cost", "purchase cost", "holding cost", "backorders cost"]):

    def avg_relative_performance(e):

        perf = {p:sum([policy[e][p][it] for it in I])/sum(base[e][p][it] for it in I) - 1 for p in P}
        perf["total"] = sum(policy[e][p][it] for p in P for it in I)/sum(base[e][p][it] for p in P for it in I) - 1

        return perf

    performances = {e:avg_relative_performance(e) for e in metrics}

    return performances


def ttest_mean_difference(sample1, sample2, sign_level = 0.05):

    t_statistic, p_value = ttest_rel(sample1, sample2)
    if p_value < sign_level: resp = 1
    else: resp = 0

    return resp

def mean_difference(base, policy, P, sign_level = 0.05, metrics = ["transport cost", "purchase cost", "holding cost", "backorders cost"]):

    significance = dict()
    for e in metrics:
        significance[e] = dict()
        for p in P+["total"]:
            significance[e][p] = ttest_mean_difference(base[e][p], policy[e][p], sign_level=sign_level)

    return significance





    

In [114]:
def get_relative_performances(params, w1 = [1,0,0,0,0], sign_level = 0.05):

    base, policy, E, K, T, I, names = get_rewards_and_sets(w1, params["w2"], theta = params["theta"], gamma = params["gamma"], min_q = params["min_q"])

    sign_perf = mean_difference(base,policy, K, sign_level = sign_level, metrics = E)
    avg_rel_perf = compute_relative_performance(base, policy, K, I, metrics = E)

    tc_base, tc_policy = compute_total_cost_differences(base, policy, E, K, I)
    tc_sign = ttest_mean_difference(tc_base, tc_policy, sign_level=sign_level)

    texts = list()

    total = sum(tc_policy)/sum(tc_base) - 1
    texts.append(f"{total:+.1%}"+"*"*tc_sign)
    for e in ["climate","water","land","fossil"]:
        texts.append(f"{avg_rel_perf[e]['total']:+.1%}"+"*"*sign_perf[e]["total"])
    
    return texts


def experiments(weights, thetas, gammas, min_qs, order = ["weights","thetas","gammas","min_qs"]):

    params = deepcopy(locals())
    iterables = [params[p] for p in order]
    combinations = product(*iterables)

    titles = {"weights":"Obj. Weights", "thetas":"Serv. Level", "gammas":"Temp. Disc.", "min_qs":"Min. Purch."}
    experiment_params = {"weights":"w2", "thetas":"theta", "gammas":"gamma", "min_qs":"min_q"}
    ordered_exp_params = [experiment_params[order[i]] for i in range(len(order))]

    offset = [31 if i == "weights" else 15 for i in order]
    print(*["-"*offset[i] for i in range(len(order))],*["-"*16 for i in range(5)],sep="-")
    print(*[" "*int(np.floor((offset[i]-len(titles[order[i]]))/2))+titles[order[i]]+" "*int(np.ceil((offset[i]-len(titles[order[i]]))/2)) for i in range(len(order))],sep="\t")
    print(*["-"*offset[i] for i in range(len(order))],*["-"*16 for i in range(5)],sep="-")

    for comb in combinations:
        params_c = dict(zip(ordered_exp_params,comb))
        texts = get_relative_performances(params_c)
        lasts = [True]; ix = len(order)-1
        for j in order[::-1]:
            if comb[ix] == params[j][-1]:
                lasts = [True] + lasts; ix -= 1
            else:
                for k in range(len(order)-len(lasts)):
                    lasts = [False] + lasts
                break
        
        text_params = {k:str(params_c[k]) if k == "w2" else f"{params_c[k]:.0%}" for k in params_c}
        offset_c = {k:15 if k != "w2" else 31 for k in params_c}
        print(*[" "*int(np.floor((offset_c[k]-len(text_params[k]))/2))+text_params[k]+" "*int(np.ceil((offset_c[k]-len(text_params[k]))/2)) for k in params_c], *[" "*int(15-len(i))+i+" " for i in texts], sep="\t")
        print(*["-"*offset[i]*lasts[i]+" "*offset[i]*(1-lasts[i]) for i in range(len(order))],*["-"*16 for i in texts],sep="-")
    

In [115]:
weights = [[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]]; thetas = [0.95, 0.9, 0.8]; gammas = [1, 0.99]; min_qs = [0.1]


experiments(weights, thetas, gammas, min_qs, order = ["min_qs","gammas","thetas","weights"])


--------------------------------------------------------------------------------------------------------------------------------------------------------------------
  Min. Purch.  	  Temp. Disc.  	  Serv. Level  	         Obj. Weights          
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
      10%      	     100%      	      95%      	        [0, 1, 0, 0, 0]        	        +21.3%* 	       +102.6%* 	        +92.9%* 	       +144.3%* 	        +82.8%* 
               -               -               ---------------------------------------------------------------------------------------------------------------------
      10%      	     100%      	      95%      	        [0, 0, 1, 0, 0]        	        +20.6%* 	       +102.4%* 	        +92.3%* 	       +145.1%* 	        +82.0%* 
               -               -               ------------------------------------------------

In [95]:
print("Florida","Wild\nOne","Sia")

Florida Wild
One Sia
